In [251]:
#bootstrap donated by Juan Carlos :)

In [250]:
%%javascript
    $("#homeButton").remove()
    $('body').append('<a href="#'+$("h1,h2,h3:eq(0)").attr("id")+'" style="position: fixed; bottom: 10px; right: 10px;" type="button" class="btn btn-success btn-circle btn-lg"><i class="glyphicon glyphicon-link" id="homeButton"></i></a>');
    $("#MainMenu").remove()
    var menu = 
      '<div id="MainMenu" style="position: fixed; top: 120px; right: 10px; z-index:6; max-height: 500px;">'+
        '<div class="list-group panel" >'+
          '<a href="#collapseMenu" class="list-group-item list-group-item-success" data-toggle="collapse" data-parent="#MainMenu">Indíce<i class="fa fa-caret-down"></i></a>'+
          '<div class="collapse" id="collapseMenu" style="overflow-y: overlay; max-height: 500px;">'+
          '</div>'+
        '</div>'+
      '</div>'
   
    var parent = $(menu)
    var arrayIds = []
    $("h1,h2,h3").attr("id",function(id,Value){
        if(Value != ""){
            var content = (Value.replace(new RegExp('-', 'g'), ' '));
            content = "&nbsp;".repeat(parseInt($(this).prop("tagName")[1])*2-1) + content;
            $(parent).find("#collapseMenu").append('<a href="#'+Value+'" style="position:relative;" class="list-group-item" data-parent="#SubMenu1">'+content+'</a>');
        }
    })
$('body').append(parent)

<IPython.core.display.Javascript object>

# Spark Notes

In order to learn Spark, as stated in https://github.com/jadianes/spark-py-notebooks by Jose A. Dianes, we are going to use 10% of the results of the KDD Cup 1999 Data Set. The reference for those (and these) spark notes is Learning Spark by Holden Karau, Andy Konwinski, Patrick Wendell, and Matei Zaharia. this version is in Python 3 :).


## KDD Cup 1999

The task for the classifier learning contest organized in conjunction with the KDD'99 conference was to learn a predictive model (i.e. a classifier) capable of distinguishing between legitimate and illegitimate connections in a computer network.

The data looks like this:

|   |     |      |    |     |      |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |      |      |      |      |      |      |      |    |    |      |      |      |      |      |      |      |      |         | 
|---|-----|------|----|-----|------|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|------|------|------|------|------|------|------|----|----|------|------|------|------|------|------|------|------|---------| 
| 0 | tcp | http | SF | 181 | 5450 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 8 | 8 | 0.00 | 0.00 | 0.00 | 0.00 | 1.00 | 0.00 | 0.00 | 9  | 9  | 1.00 | 0.00 | 0.11 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | normal. | 
| 0 | tcp | http | SF | 239 | 486  | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 8 | 8 | 0.00 | 0.00 | 0.00 | 0.00 | 1.00 | 0.00 | 0.00 | 19 | 19 | 1.00 | 0.00 | 0.05 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | normal. | 
| 0 | tcp | http | SF | 235 | 1337 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 8 | 8 | 0.00 | 0.00 | 0.00 | 0.00 | 1.00 | 0.00 | 0.00 | 29 | 29 | 1.00 | 0.00 | 0.03 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | normal. | 
| 0 | tcp | http | SF | 219 | 1337 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 6 | 6 | 0.00 | 0.00 | 0.00 | 0.00 | 1.00 | 0.00 | 0.00 | 39 | 39 | 1.00 | 0.00 | 0.03 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | normal. | 



# Getting data in Spark or RDD creation

Apache Spark data structure:

- **RDD (Resilient Distributed Datasets)**: Distributed collection of elements 

All work in Spark is expressed as:

- **Creating** new RDDs
- **Transforming** existing RDDs
- **Calling actions** on RDDs to compute a result

The most common way of creating an RDD is to load it from a file. 

**Spark's textFile** can handle compressed files directly.

## Method a) Creating RDD from a file

In [1]:
#Initiallizing Spark (just run once)
import pyspark
sc = pyspark.SparkContext('local[*]')
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [2]:
#Retrieve data (don't forget to add the retrieve)
import urllib.request
f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

#Loading data
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

In [3]:
#Count the number of lines in the file 
raw_data.count()

494021

In [4]:
#Check first few entries
raw_data.take(5)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.']

## Method b) Creating RDD from a list using parallelize

In [5]:
# Existing list
a = range(100)

data = sc.parallelize(a)

In [6]:
# Count the number of elements on RDD
data.count()

100

In [7]:
# Access the first elements
data.take(5)

[0, 1, 2, 3, 4]

# Basic transformations and actions

There are two basic classes of distributed unary (only one input) operations for RDD's, transformations and actions. 

- **The transformations** are distributed operations whose result generates a **_new RDD from another_**. they are **lazy**, because they are not performed until an action is called.
- **The actions** are operations that apply a function of aggregation (such as mean, median,...) or others in a **_systematic and recurrent way to all the RDD_**. Return a value other than an RDD. Is performed immediatly.

## Python's lambdas

Python supports the creation of anonymous functions (i.e. functions that are not bound to a name) at runtime, using a construct called *lambda*. 

In [8]:
g = lambda x: x**2
print(g(8))

64


## Transformations

Transformations are distributed operations whose result generates a new RDD from another

### `filter` 

Applied to RDDs to keep only elements that satisfy certain conditions

In [9]:
# Example: counting normal interactions in the dataset
## Consider you are not passing arrays but strings, so finding the pattern 'normal.' is straightforward.
normal_raw_data = raw_data.filter(lambda x: 'normal.' in x)

In [10]:
# How many elements are in the new RDD?
from time import time
t0 = time()
normal_count = normal_raw_data.count()
tt = time() - t0
print ("There are {} 'normal' interactions".format(normal_count))
print ("Count completed in {} seconds".format(round(tt,3)))

There are 97278 'normal' interactions
Count completed in 0.827 seconds


From a total of 494021 in our 10 percent dataset, 97278 contain the *normal* tag word.

### `map`

Map applies a function to every element in the RDD. In general Python's lambdas are useful for this.

In [11]:
# To read a file as a .csv formated one:
from pprint import pprint
csv_data = raw_data.map(lambda x: x.split(","))
#csv_data=raw_data.map(lambda x: str(x)[0:3])
t0 = time()
head_rows = csv_data.take(5)
tt = time() - t0
print ("Parse completed in {} seconds".format(round(tt,3)))

#print("{}".format(head_rows))

#pprint(head_rows[4])

Parse completed in 0.023 seconds


In [12]:
t0 = time()
head_rows = csv_data.take(100000)
tt = time() - t0
print ("Parse completed in {} seconds".format(round(tt,3)))

Parse completed in 1.165 seconds


#### `map` with predefined functions

`map` can be used with predefined functions.

In [13]:
# Have each element as a key-value par where the key is the tag ("normal." in this case in column 41)

def parse_interaction(line):
    elems = line.split(",")
    tag = elems[41]
    return (tag,elems)

key_csv_data = raw_data.map(parse_interaction)
head_rows = key_csv_data.take(5)
#pprint(head_rows[0])

### `sample`

Takes up to three parameters:

1. Whether the sampling is done with replacement (True) or not (False). 
2. Second is the sample size as a fraction. 
3. Finally we can optionally provide a random seed.

In [14]:
# Example

raw_data_sample = raw_data.sample(False, 0.1, 1234)
sample_size = raw_data_sample.count()
total_size = raw_data.count()
print ("Sample size is {} of {}".format(sample_size,total_size))

Sample size is 49493 of 494021


In [15]:
# Naive example: Approximation of the proportion of "normal." with the .csv data
##csv_data = raw_data.map(lambda x: x.split(","))
raw_data_sample_items = raw_data_sample.map(lambda x: x.split(","))
sample_normal_tags = raw_data_sample_items.filter(lambda x: "normal." in x)

# actions + time
t0 = time()
sample_normal_tags_count = sample_normal_tags.count()
tt = time() - t0

sample_normal_ratio = sample_normal_tags_count / float(sample_size)
print ("The ratio of 'normal' interactions is {}".format(round(sample_normal_ratio,3)) )
print ("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.195
Count done in 0.911 seconds


In [16]:
#Continuation of naive example: Calculation without sampling (whole data)
#Separating by , and filtering the normal ones
raw_data_items = raw_data.map(lambda x: x.split(","))
normal_tags = raw_data_items.filter(lambda x: "normal." in x)

# actions + time
t0 = time()
normal_tags_count = normal_tags.count()
tt = time() - t0

normal_ratio = normal_tags_count / float(total_size)
print ("The ratio of 'normal' interactions is {}".format(round(normal_ratio,3)))
print ("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.197
Count done in 1.657 seconds


The more transformations and actions we apply after the sampling the bigger this gain vs using whole data. This is because without sampling all the transformations are applied to the complete set of data.


#### Sample vs **takeSample** action.

If what we need is to grab a sample of raw data from our RDD into local memory in order to be used by other non-Spark libraries, `takeSample` can be used.

### `ReducebyKey`

In [124]:
#Count or do something given a provided key in tuples
people = sc.textFile("people.txt")
people=people.map(lambda x: x.split('\t'))
people.map(lambda t:(t[1],1)).reduceByKey(lambda x,y:x+y).collect()

[('M', 3), ('F', 3)]

## Actions

Are operations that apply a function of aggregation (such as mean, median,...) or others in a systematic and recurrent way to all the RDD.

### `Collect`

Gets all elements in the RDD into memory. Be careful with large RDDs.

In [17]:
#Example

t0 = time()
all_raw_data = raw_data.collect()
tt = time() - t0
print ("Data collected in {} seconds".format(round(tt,3)))

Data collected in 2.557 seconds


Every Spark worker node that has a fragment of the RDD has to be coordinated in order to retrieve its part, and then *reduce* everything together.

In [18]:
# So to collect in memory all the normal interactions as key-value pairs:

## get data from file
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

# parse into key-value pairs with the previously created parse_interaction function in the map section
##def parse_interaction(line):
  ##  elems = line.split(",")
  ##  tag = elems[41]
  ##  return (tag,elems)

key_csv_data = raw_data.map(parse_interaction)

# collect all normal as defined in the previous filter
## normal_raw_data = raw_data.filter(lambda x: 'normal.' in x)
normal_key_interactions = key_csv_data.filter(lambda x: x[0] == "normal.")

t0 = time()
all_normal = normal_key_interactions.collect()
tt = time() - t0
normal_count = len(all_normal)
print ("Data collected in {} seconds".format(round(tt,3)))
print ("There are {} 'normal' interactions".format(normal_count))

Data collected in 2.354 seconds
There are 97278 'normal' interactions


#### `.count` vs `collect + len`  

- **`collect + len`** matches with the previous count for normal interactions. 
- **`collect + len`** is more time consuming because we retrieve all the data and then use Python's len on the resulting list
- with **`count`** we were just counting the total number of elements in the RDD.

### `Reduce`

### `takeSample`

Grab a sample of raw data from our RDD into local memory in order to be used by other non-Spark libraries.

Characteristics:

- Sintaxis very similar to sample, but you specify the number of items instead of the sample size:
    1. Whether the sampling is done with replacement (True) or not (False). 
    2. Second is the number of items to be chosen. 
    3. Finally we can optionally provide a random seed. 

In [20]:
# Take a sample for other libraries
t0_1 = time()
raw_data_sample = raw_data.sample(False, 0.1, 1234)
#raw_data_sample_items = raw_data_sample.map(lambda x: x.split(","))
#sample_normal_tags = raw_data_sample_items.filter(lambda x: "normal." in x)
tt_1 = time() - t0_1


t0_2 = time()
raw_data_sample = raw_data.takeSample(False, 400000, 1234)
normal_data_sample = [x.split(",") for x in raw_data_sample if "normal." in x]
tt_2 = time() - t0_2

normal_sample_size = len(normal_data_sample)

normal_ratio = normal_sample_size / 400000.0
print ("The ratio of 'normal' interactions is {}".format(normal_ratio))
#print ("Assignation, splitting and filtering done in {} seconds with sample, vs {} seconds with takeSample".format(round(tt_1,3),round(tt_2,3)))
print ("Assignation of sample done in {} seconds with sample, vs {} seconds with takeSample".format(round(tt_1,3),round(tt_2,3)))

The ratio of 'normal' interactions is 0.1967175
Assignation of sample done in 0.034 seconds with sample, vs 2.329 seconds with takeSample


It took longer, even with a slightly smaller sample. The reason is that Spark just distributed the execution of the sampling process. The filtering and splitting of the results were done locally in a single node.

### `first`

Gives you the first element of an RDD

In [64]:
sc.parallelize(range(1,10)).first()

1

# Excercises from https://www.youtube.com/watch?v=9xYfNznjClE

Given a list:

In [54]:
a = [(1,2),(2,4),(5,6)]
data = sc.parallelize(a)

Write an expression to get only the second elements of each tupple:

In [55]:
#Remember that map is a methos of a RDD
data2 = data.map(lambda x:x[1])
print(data2.take(3))

[2, 4, 6]


Write an expression to get the sum of the second elements

In [50]:
data2.reduce(lambda y,z:y + z)

12

Write an expression to get the sum of the odd first elements

In [60]:
data1 = data.map(lambda x:x[0])
data1_1=data1.map(lambda y: y%2==1)
#If you use map instead of filter it won't do what the instruction indicated, 
# because, even if it will find which one  
print(data1_1.count())
print(data1_1.reduce(lambda b, c: b +c))

3
2


The same as the previous but with filter

In [58]:
data3 = data.map(lambda x:x[0])
#data1.map(lambda y: y%2==1)

data4 = data3.filter(lambda y: y%2==1)
print(data4.count())
print(data4.reduce(lambda b, c: b +c))

2
6


In [62]:
sc.parallelize(range(1,10)).first()

1

In [76]:
t0 = time()
list2=sc.parallelize( range(1,1000) ).map(lambda x: x*10)
tt = time() - t0
#it does nothing, that is why it seems so fats, it is lazzzzzy u.u 
t0 = time()
list2.first()
tt2 = time() - t0
print("Lazy took {} seconds vs active which took {} seconds. Which is {} times fold".format(tt,tt2,tt2/tt))

Lazy took 0.0025794506072998047 seconds vs active which took 0.017164945602416992 seconds. Which is 6.654496718735558 times fold


In [79]:
t0 = time()
list2.reduce(lambda x,y : x + y )
tt = time() - t0 
print("Reduce took {} seconds".format(tt))

Reduce took 0.03945279121398926 seconds


In [82]:
list2.filter(lambda x : x%100==0).take(3)

#list2.filter(lambda x : x%100==0).collect()

[100, 200, 300]

In [90]:
rdd1=sc.parallelize( range(1,100) )
#rdd1.map(lambda x: x*x).sum()
#rdd1.map(lambda x: x*x).max()
#rdd1.map(lambda x: x*x).min()
rdd1.map(lambda x: x*x).mean()

3316.6666666666665

## Excercises
1. Get an RDD with numbers 1 to 10
2. Get all the elements in that RDD which are divisible by 3
3. Get the product of the elements in 2

In [95]:
rdd3=sc.parallelize( range(1,10) )
#rdd3.filter(lambda x: x%3 == 0).take(3)
rdd3.filter(lambda x: x%3 == 0).reduce(lambda x,y: x*y)

162

In [96]:
sc.parallelize( range(1,10) ).filter(lambda x: x%3 == 0).reduce(lambda x,y: x*y)

162

## More excercises (about reading files)
The data comes from https://github.com/okaram/spark-pycon15/blob/master/data/people.txt

In [147]:
people = sc.textFile("people.txt")
people.take(2)

['Orlando\tM\t40\tPython', 'Lina\tF\t39\tC#']

In [148]:
#Split lines in people by tabs and take the first line
people.map(lambda x: x.split('\t')).first()

['Orlando', 'M', '40', 'Python']

In [149]:
people=people.map(lambda x: x.split('\t'))
people.take(3)

[['Orlando', 'M', '40', 'Python'],
 ['Lina', 'F', '39', 'C#'],
 ['John', 'M', '30', 'Python']]

### Keys

In [150]:
#Define keys by making dummy tuples
#people.map(lambda t:(t[1],1)).take(1)
people.map(lambda t:(t[1],1)).take(1)


[('M', 1)]

In [151]:
#people.map(lambda t:(t[1],1)).take(1)
people.map(lambda t:(t[1],1)).reduceByKey(lambda x,y:x+y).collect()

[('M', 3), ('F', 3)]

In [152]:
#Youngest per gender
#we use int to avoid interpretation as string
people.map(lambda t:(t[1],int(t[2]))).collect()

[('M', 40), ('F', 39), ('M', 30), ('F', 32), ('F', 18), ('M', 20)]

In [153]:
people.map(lambda t:(t[1],int(t[2]))).reduceByKey(lambda x,y: min(x,y)).collect()

[('M', 20), ('F', 18)]

In [154]:
people.map(lambda t:(t[1],int(t[2]))).reduceByKey(lambda x,y: max(x,y)).collect()

[('M', 40), ('F', 39)]

In [165]:
# Importing my python code
import person
people = sc.textFile("people.txt").map(lambda l: person.Person().parse(l))
people.take(3)

[Person( Orlando, gender=M, 40 years old, likes Python),
 Person( Lina, gender=F, 39 years old, likes C#),
 Person( John, gender=M, 30 years old, likes Python)]

In [168]:
#Once having done this, we can use much more informative tags
people.map(lambda t:(t.gender,t.age)).reduceByKey(lambda x,y: min(x,y)).collect()

[('M', 20), ('F', 18)]

In [171]:
people.map(lambda t:(t.gender,1)).reduceByKey(lambda x,y: x+y).collect()

[('M', 3), ('F', 3)]

## Sales example

In [184]:
# with * hadoop can do the file replacement
sales=sc.textFile("sales_*.txt").map(lambda x:x.split('\t'))
sales.collect()

[['2014-01-01', '1', '1', '100'],
 ['2014-01-01', '1', '2', '37'],
 ['2014-01-01', '1', '3', '54'],
 ['2014-01-01', '2', '1', '50'],
 ['2014-01-01', '2', '2', '40'],
 ['2014-01-01', '3', '1', '75'],
 ['2014-01-01', '4', '4', '1'],
 ['2014-01-02', '1', '1', '10'],
 ['2014-01-02', '1', '2', '31'],
 ['2014-01-02', '1', '3', '5'],
 ['2014-01-02', '2', '1', '52'],
 ['2014-01-02', '2', '2', '42'],
 ['2014-01-02', '3', '2', '72']]

In [185]:
sales.map(lambda x:int(x[3])).collect()

[100, 37, 54, 50, 40, 75, 1, 10, 31, 5, 52, 42, 72]

In [186]:
sales.map(lambda x:int(x[3])).reduce(lambda x,y:x+y)

569

## Example join

In [194]:
states=[("AL","Alabama"),
       ("AK","Alaska"),
       ("AR","Arizona")
       ];
populations=[("AL",4779736),
       ("AK",710231),
       ("AR",6392017)
       ];

states_rdd=sc.parallelize(states)
populations_rdd=sc.parallelize(populations)
states_rdd.join(populations_rdd).collect()

[('AK', ('Alaska', 710231)),
 ('AR', ('Arizona', 6392017)),
 ('AL', ('Alabama', 4779736))]

# Data Table or Data Frame

- Knows its fields, has no methods
- Similar to a data table in pandas
- All rows must have the same fields
- Queries can be done such as in RDDs or through SQL


## Reading a .json as datatable (and its methods)


In this case, several methods can be used:

- .select: like map, can use strings or columns
```python
people.select("name"people.age+1).show()
```
- .filter: can filter certain rows
```python
people.filter(people.age>30).show()
```
- .where: similar to filter

- .show: displays nicely
- Use pandas syntax for filter
```python
people[people.gender=='F']
```
- GroupBy: returns a filtered RDD
```python
people.groupBy(people.gender]).count()
```

In [204]:
# Needs to use sqlCtx.jsonFile sql Context

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

people=sqlContext.read.json("people.json")

#Terminal friendly
#people.show()

#User friendly aspect in jupyter (also terminal friendly)
people.limit(10).toPandas()

#people.collect()

,age,favorite_language,gender,name
0,40,Python,M,Orlando
1,39,C#,F,Lina
2,30,Python,M,John
3,32,Python,F,Jane
4,18,Python,F,Michelle
5,20,C#,M,Daniel


In [206]:
people.first()

Row(age=40, favorite_language='Python', gender='M', name='Orlando')

### .select (similar to lambda but not tht cool)

In [228]:
people.select("name").show()

+--------+
|    name|
+--------+
| Orlando|
|    Lina|
|    John|
|    Jane|
|Michelle|
|  Daniel|
+--------+



In [208]:
people.select("name",people.age+1).show()

+--------+---------+
|    name|(age + 1)|
+--------+---------+
| Orlando|       41|
|    Lina|       40|
|    John|       31|
|    Jane|       33|
|Michelle|       19|
|  Daniel|       21|
+--------+---------+



### .filter

In [215]:
%%time

people.filter(people.age>30).show()

+---+-----------------+------+-------+
|age|favorite_language|gender|   name|
+---+-----------------+------+-------+
| 40|           Python|     M|Orlando|
| 39|               C#|     F|   Lina|
| 32|           Python|     F|   Jane|
+---+-----------------+------+-------+

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 35.9 ms


### .where

In [216]:
%%time

people.where(people.age>30).show()

+---+-----------------+------+-------+
|age|favorite_language|gender|   name|
+---+-----------------+------+-------+
| 40|           Python|     M|Orlando|
| 39|               C#|     F|   Lina|
| 32|           Python|     F|   Jane|
+---+-----------------+------+-------+

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 40.1 ms


### Use pandas syntax for filter

In [218]:
people[people.gender=='F'].show()

+---+-----------------+------+--------+
|age|favorite_language|gender|    name|
+---+-----------------+------+--------+
| 39|               C#|     F|    Lina|
| 32|           Python|     F|    Jane|
| 18|           Python|     F|Michelle|
+---+-----------------+------+--------+



#### Two pandas filter in one

In [247]:
people[people.age>30][people.gender=="F"].show()

+---+-----------------+------+----+
|age|favorite_language|gender|name|
+---+-----------------+------+----+
| 39|               C#|     F|Lina|
| 32|           Python|     F|Jane|
+---+-----------------+------+----+



In [249]:
people[(people.age>30) & (people.gender=="F")].show()

+---+-----------------+------+----+
|age|favorite_language|gender|name|
+---+-----------------+------+----+
| 39|               C#|     F|Lina|
| 32|           Python|     F|Jane|
+---+-----------------+------+----+



### GroupBy  (returns a grouped RDD)

In [229]:
people.groupBy(people.gender).count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|    3|
|     M|    3|
+------+-----+



In [231]:
people.groupBy(people.gender).mean('age').show()

+------+------------------+
|gender|          avg(age)|
+------+------------------+
|     F|29.666666666666668|
|     M|              30.0|
+------+------------------+



## Use table as SQL

1. Register as TempTable

In [233]:
people.registerTempTable("people")

In [234]:
sqlContext.sql("select name, age FROM people").show()

+--------+---+
|    name|age|
+--------+---+
| Orlando| 40|
|    Lina| 39|
|    John| 30|
|    Jane| 32|
|Michelle| 18|
|  Daniel| 20|
+--------+---+



In [235]:
sqlContext.sql("select gender, avg(age) AS AvgAge FROM people GROUP BY gender").show()

+------+------------------+
|gender|            AvgAge|
+------+------------------+
|     F|29.666666666666668|
|     M|              30.0|
+------+------------------+



# Performance

- Spark in python is slower than scala due to translation
- Datatable (dataFrame) avoids this translation (all lives in JVm until the last step to client)
- Datatable should be able to optimize better, but you lose control
- Join and reduce are very expensive

Spark keeps everything in RAM, if you need the RAM for something else, spark kills the part used and recalculates each time :).


# Resources:
    
- [Markdown Cheat sheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)
- [Lambda functions in python](http://www.secnetix.de/olli/Python/lambda_functions.hawk)
- [Introduction to Spark with python](https://www.youtube.com/watch?v=9xYfNznjClE)